In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from circuit import Circuit
import numpy as np
import dgl.function as fn
import random
import networkx as nx
from utils import *
from dgl.data.utils import save_graphs, load_graphs
import time

Using backend: pytorch


In [2]:
import os.path
preprocess_begin = time.time()

design = "ldpc_GNN"
path = design+"/heterograph.bin"

cir = Circuit(design)
root = "/home/sh528/M3Ddesigns/"+design+"/"
cir.parseHierVerilog(root+"die0.v")
cir.parseHierVerilog(root+"die1.v")
cir.parseTop(root+"top.v")
# cir.parseVerilog(design+"/"+design+".v")
# cir.parsePartition(design+"/die0.rpt")
stil = design+"/TDF.stil"

# if False:
if os.path.isfile(path):
    n_patterns = cir.parseSTIL(stil, -2)
    dic, topEdge = backprop(cir)
    hg = load_graphs("./"+design+"/heterograph.bin")[0][0]
else:
    n_patterns = cir.parseSTIL(stil)
    edge = CreateGraphByFaultSite(cir)
    dic, topEdge = backprop(cir)

    hg = dgl.heterograph({ ('topNode', 'topEdge', 'faultSite'): topEdge, ('faultSite', 'net', 'faultSite'): edge })
    feats = torch.tensor([cir.Node[n].net.feats for n in cir.Node])
    hg.nodes['faultSite'].data['feats'] = feats
    hg.nodes['faultSite'].data['in_degree'] = hg.in_degrees(etype='net').view(-1,1).float()
    hg.nodes['faultSite'].data['out_degree'] = hg.out_degrees(etype='net').view(-1,1).float()
    hg.nodes['faultSite'].data['top_degree'] = hg.in_degrees(etype='topEdge').view(-1,1).float()
    hg.nodes['faultSite'].data['level'] = getLevel(cir)
    hg.nodes['faultSite'].data['loc'] = getLocation(cir, hg.num_nodes('faultSite'))
    hg.nodes['faultSite'].data['more'] = addfeatures(cir, hg.num_nodes('faultSite'))
    save_graphs("./"+design+"/heterograph.bin", hg)
    
print(hg)

Start parsing verilog netlist
nodeID: 114029
End parsing verilog netlist
CPU time: 8.29s

Start parsing verilog netlist
nodeID: 227962
End parsing verilog netlist
CPU time: 9.63s

Start parsing top verilog netlist
End parsing verilog netlist
CPU time: 2.89s

Start parsing STIL patterns
Pass Pattern 0
Final Pat
End parsing STIL patterns
CPU time: 5.82s

Graph(num_nodes={'faultSite': 292654, 'topNode': 2048},
      num_edges={('faultSite', 'net', 'faultSite'): 356289, ('topNode', 'topEdge', 'faultSite'): 7274880},
      metagraph=[('faultSite', 'faultSite', 'net'), ('topNode', 'faultSite', 'topEdge')])


In [3]:
print(torch.sum(hg.nodes['faultSite'].data['loc'], dim=0))

tensor([114029., 113933.,  64692.])


In [7]:
start_pat = 186
end_pat = 372

dataset, dstIDset = getDatasetfromLog(cir, design, dic, hg, n_patterns, 10, start_pat, end_pat)
subgraphs = getSubgraphs(hg, dataset, dstIDset, True, start_pat, end_pat)
print("Number of samples: {}".format(len(subgraphs)))


preprocess_end = time.time()

print("Total CPU time for preprocessing: {}".format(preprocess_end-preprocess_begin))

Start generating data
Finish generating data
Start generating subgraphs
End generating subgraphs
Number of samples: 10
Total CPU time for preprocessing: 232.10093212127686


In [9]:
print(subgraphs[1][0])

Graph(num_nodes=144, num_edges=269,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), '_TYPE': Scheme(shape=(), dtype=torch.int64), 'infeats': Scheme(shape=(11,), dtype=torch.float32)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), '_TYPE': Scheme(shape=(), dtype=torch.int64)})


In [ ]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
#         self.lin1 = nn.Linear(11,32)
        self.conv1 = GraphConv(11, 32)
#         self.conv2 = GraphConv(128, 32)
        self.conv3 = GraphConv(32, 3)
#         self.norm1 = nn.BatchNorm1d(128)
#         self.norm2 = nn.BatchNorm1d(32)
#         self.classify1 = nn.Linear(3, 32)
#         self.classify2 = nn.Linear(64, 3)
        

    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
#         h = self.norm1(h)
#         h = self.conv2(g, h)
#         h = F.relu(h)
#         h = self.norm2(h)
        h = self.conv3(g, h)
#         h = F.relu(h)
        g.ndata['h_final'] = h
        hg = dgl.mean_nodes(g, 'h_final')
        
        return hg
#         hg = F.softmax(hg, dim=0)
#         ratio = torch.sum(in_feat[:, 3:6], dim=0)
#         ratio = self.classify1(ratio)
#         hg = torch.cat([hg, ratio.unsqueeze(dim=0)], dim=1)
# #         print(hg)
        
# #         hg = self.classify1(hg)
# #         hg = F.relu(hg)
        
#         return self.classify2(hg)


In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
#         self.lin1 = nn.Linear(11,32)
        self.gcn1 = GCN()
        self.gcn2 = GCN()
        self.gcn3 = GCN()
#         self.norm1 = nn.BatchNorm1d(128)
#         self.norm2 = nn.BatchNorm1d(32)
#         self.classify1 = nn.Linear(3, 32)
#         self.classify2 = nn.Linear(64, 3)
        

    
    def forward(self, g, in_feat):
        
        outputs = []
        outputs.append(F.softmax(self.gcn1(g, in_feat), dim=1))
        outputs.append(F.softmax(self.gcn2(g, in_feat), dim=1))
        outputs.append(F.softmax(self.gcn3(g, in_feat), dim=1))
        
        outputs = torch.cat([F.softmax(self.gcn1(g, in_feat), dim=1), 
                             F.softmax(self.gcn2(g, in_feat), dim=1), 
                             F.softmax(self.gcn3(g, in_feat), dim=1)]
                            , dim=0)
        
#         print(outputs)
#         print(torch.mean(outputs,0))
#         print(torch.mean(outputs,0).unsqueeze(dim=0))
        
        return torch.mean(outputs,0).unsqueeze(dim=0)

In [ ]:
num_examples = len(subgraphs)
num_train = int(num_examples * 0.7)
num_val = int(num_examples * 0.15)

random.shuffle(subgraphs)

# # Random
train_sampler = SubsetRandomSampler(torch.arange(num_train))
val_sampler = SubsetRandomSampler(torch.arange(num_train, num_train+num_val))
test_sampler = SubsetRandomSampler(torch.arange(num_train+num_val, num_examples))

train_dataloader = GraphDataLoader(
    subgraphs, sampler=train_sampler, batch_size=1, drop_last=False)
val_dataloader = GraphDataLoader(
    subgraphs, sampler=val_sampler, batch_size=1, drop_last=False)
test_dataloader = GraphDataLoader(
    subgraphs, sampler=test_sampler, batch_size=1, drop_last=False)


In [ ]:
train_begin = time.time()

model = Model().to('cuda')
# g = g.to('cpu')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

train_loss_values = []
val_loss_values = []
train_acc_values = []
val_acc_values = []

for epoch in range(50):
    print("\nEpoch %d:" %epoch)
    print("Training...")
    model.train()
    correct = []
    incorrect = []

    train_acc = 0
    train_loss = 0
    num_tests = 0
    train_error = np.zeros((3,3))
    for g, l in train_dataloader:
        g = g.to('cuda')
        labels = l.to('cuda')
#         g = dgl.add_reverse_edges(g)
#         g = dgl.add_self_loop(g)

        infeats = g.ndata['infeats']
        


        pred = model(g, infeats)
#         print(pred.shape)
        loss = F.cross_entropy(pred, labels)
#         l1_lambda = 0.001
#         l1_norm = sum(p.abs().sum() for p in model.parameters())
#         loss = loss + l1_lambda * l1_norm
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_acc += (pred.argmax(1) == labels).sum().item()
        train_loss += loss
        num_tests += len(labels)
        for i in range(len(labels)):
                p = pred[i]
                l = labels[i]
                train_error[p.argmax(0).item()][l.item()] += 1

    avg_loss = train_loss/num_tests
    avg_acc = train_acc/num_tests
    train_loss_values.append(avg_loss)
    train_acc_values.append(avg_acc)

    print('In epoch {}, train loss: {:.3f}, train acc: {:.3f}'.format(epoch, avg_loss, avg_acc))
    print(train_error)
#     print("Mean correct: {}, Mean incorrect: {}".format(np.mean(correct), np.mean(incorrect)))

    print("Validation...")
    val_error = np.zeros((3,3))
    model.eval()
    correct = []
    incorrect = []
    val_acc = 0
    val_loss = 0
    num_tests = 0

#     random.shuffle(val_set)
    with torch.no_grad():
        for g, l in val_dataloader:
            g = g.to('cuda')
            labels = l.to('cuda')
#             g = dgl.add_reverse_edges(g)
#             g = dgl.add_self_loop(g)

            infeats = g.ndata['infeats']

            pred = model(g, infeats)
            loss = F.cross_entropy(pred, labels)
            optimizer.zero_grad()

            val_acc += (pred.argmax(1) == labels).sum().item()

            val_loss += loss
            num_tests += len(labels)

            for i in range(len(labels)):
                p = pred[i]
                l = labels[i]
                val_error[p.argmax(0).item()][l.item()] += 1
            
    avg_loss = val_loss/num_tests
    avg_acc = val_acc/num_tests
    val_loss_values.append(avg_loss)
    val_acc_values.append(avg_acc)

    print('In epoch {}, val loss: {:.3f}, val acc: {:.3f}'.format(epoch, avg_loss, avg_acc))
    print(val_error)
#     print("Mean correct: {}, Mean incorrect: {}".format(np.mean(correct), np.mean(incorrect)))
#     scheduler.step()

#     
train_end = time.time()
print("\nTraining time for {} epochs: {}\n".format(epoch+1, train_end-train_begin))
    
model.eval()
test_acc = 0
test_loss = 0
num_tests = 0
test_error = np.zeros((3,3))
test_dist = np.zeros((3,3))

with torch.no_grad():
    for g, l in test_dataloader:
        g = g.to('cuda')
        labels = l.to('cuda')
#         g = dgl.add_reverse_edges(g)
#         g = dgl.add_self_loop(g)
        infeats = g.ndata['infeats']

        pred = model(g, infeats)
        loss = F.cross_entropy(pred, labels)
        optimizer.zero_grad()

        test_acc += (pred.argmax(1) == labels).sum().item()
        test_loss += loss
        num_tests += len(labels)
        
        for i in range(len(labels)):
                p = pred[i]
                l = labels[i]
                test_error[p.argmax(0).item()][l.item()] += 1
                smax = F.softmax(p, dim=0)
                test_dist[p.argmax(0).item()][l.item()] += abs(smax[p.argmax(0).item()].item() - smax[l.item()].item())


avg_loss = test_loss/num_tests
avg_acc = test_acc/num_tests

print("test accuracy: {}".format(avg_acc))
print(test_error)
print(test_dist/test_error)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(0,len(train_loss_values),1), train_loss_values,'b', np.arange(0,len(val_loss_values),1), val_loss_values, 'g')
plt.legend(["Train", "Validation"])
plt.title("Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

plt.plot(np.arange(0,len(train_acc_values),1), train_acc_values,'b', np.arange(0,len(val_acc_values),1), val_acc_values, 'g')
# plt.plot(np.arange(0,25,1), train_acc_values[-25:],'b', np.arange(0,25,1), val_acc_values[-25:], 'g')

plt.legend(["Train", "Validation"])
plt.title("Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()


In [ ]:
model.eval()
test_acc = 0
test_loss = 0
num_tests = 0
test_error = np.zeros((3,3))
test_dist = np.zeros((3,3))
graphs = []
g2 = []
gl = []

with torch.no_grad():
    for g, l in test_dataloader:
        g = g.to('cuda')
        labels = l.to('cuda')
#         g = dgl.add_reverse_edges(g)
#         g = dgl.add_self_loop(g)
        infeats = g.ndata['infeats']
        ubg = dgl.unbatch(g)

        pred = model(g, infeats)
        loss = F.cross_entropy(pred, labels)
        optimizer.zero_grad()

        test_acc += (pred.argmax(1) == labels).sum().item()
        test_loss += loss
        num_tests += len(labels)
        
        for i in range(len(labels)):
            p = pred[i]
            ll = labels[i]
            
            if p.argmax(0) != ll and p.argmax(0).item() == 2:
                graphs.append(ubg[i].to('cpu'))
                gl.append(ll.to('cpu'))
            elif p.argmax(0) == ll and p.argmax(0).item() == 2:
                g2.append(ubg[i].to('cpu'))

avg_loss = test_loss/num_tests
avg_acc = test_acc/num_tests

print("test accuracy: {}".format(avg_acc))
print(test_error)
print(test_dist/test_error)

In [ ]:
for j in range(len(graphs)):
    g1 = graphs[j]
    G = dgl.to_networkx(g1)
    color_map = []
    for i in range(g1.num_nodes()):
        if g1.ndata['infeats'][i][3] == 1:
            color_map.append('blue')
        elif g1.ndata['infeats'][i][4] == 1:
            color_map.append('green')
        elif g1.ndata['infeats'][i][5] == 1:
            color_map.append('red')
    nx.draw(G, node_color=color_map, with_labels=True)
    print(gl[j])
    plt.show()

In [ ]:
for j in range(len(g2)):
    g1 = g2[j]
    G = dgl.to_networkx(g1)
    color_map = []
    for i in range(g1.num_nodes()):
        if g1.ndata['infeats'][i][3] == 1:
            color_map.append('blue')
        elif g1.ndata['infeats'][i][4] == 1:
            color_map.append('green')
        elif g1.ndata['infeats'][i][5] == 1:
            color_map.append('red')
    nx.draw(G, node_color=color_map, with_labels=True)
#     print(gl[j])
    plt.show()